In [29]:
import torch
import numpy as np
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [30]:
TRAIN_DATA_PATH = "train"
TEST_DATA_PATH = "test"
training_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform = ToTensor())
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform = ToTensor())


In [31]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [32]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 3, 48, 48])
Shape of y: torch.Size([128]) torch.int64


In [33]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=3,              
                out_channels=64,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(64, 128, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(128, 256, 3, 1, 1),     
            nn.ReLU(),                                      
        )
        self.conv4 = nn.Sequential(         
            nn.Conv2d(256, 256, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # for 6th layer
        self.conv5 = nn.Sequential(         
            nn.Conv2d(256, 128, 3, 1, 1),     
            nn.ReLU(), 
        )

        self.conv6 = nn.Sequential(         
            nn.Conv2d(128, 64, 3, 1, 1),     
            nn.ReLU(), 
        )

        # for 7th layer
        self.conv7 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, 1),     
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        self.dropout = nn.Dropout(0.3)
        # fully connected layer


        self.out = nn.Sequential(
            # change accordingly to number of layers
            nn.Linear(6*6*32,7),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.dropout(x)
        # flatten 
        x = x.view(x.size(0), -1)      
        output = self.out(x)
        return output

model = CNN().to(device)
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv6): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv7): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(

In [34]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    pbar = tqdm(total=28709//128)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        pbar.update(1)
    pbar.close()


In [36]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
epochs = 64
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


  0%|          | 0/224 [00:00<?, ?it/s]

loss: 1.906894  [    0/28709]
loss: 1.369783  [12800/28709]
loss: 3.535709  [25600/28709]
Test Error: 
 Accuracy: 17.4%, Avg loss: 1.977964 

Epoch 2
-------------------------------


  0%|          | 0/224 [00:00<?, ?it/s]

loss: 2.225109  [    0/28709]
loss: 0.100310  [12800/28709]
loss: 3.700470  [25600/28709]
Test Error: 
 Accuracy: 17.4%, Avg loss: 2.115003 

Epoch 3
-------------------------------


...

  0%|          | 0/224 [00:00<?, ?it/s]

loss: 2.043719  [    0/28709]
loss: 1.845674  [12800/28709]
loss: 6.652040  [25600/28709]
Test Error: 
 Accuracy: 11.6%, Avg loss: 3.103937 

Epoch 23
-------------------------------


  0%|          | 0/224 [00:00<?, ?it/s]

loss: 2.087676  [    0/28709]


KeyboardInterrupt: 

In [16]:
torch.save(model.state_dict(), "model 7 layer 55 acc.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [37]:
model = CNN()
model.load_state_dict(torch.load("model 7 layer 55 acc.pth"))
model.to(device)



RuntimeError: Error(s) in loading state_dict for CNN:
	size mismatch for conv3.0.weight: copying a param with shape torch.Size([64, 128, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 128, 3, 3]).
	size mismatch for conv3.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for conv4.0.weight: copying a param with shape torch.Size([64, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 256, 3, 3]).
	size mismatch for conv4.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([256]).

In [24]:
print('cnn')
test(test_dataloader, model, loss_fn)

cnn


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same